Detección de caras con webcam

In [2]:
import cv2
import dlib
import numpy as np

En el bucle de procesamiento, la tecla 'd' permite cambiar entre filtros

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [4]:
# Inicializar el detector de caras y el predictor de landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Cargar las imágenes de la peluca de Goku en estado normal y en Super Saiyan con canal alfa
goku_base = cv2.imread("goku_base1.png", -1)
goku_saiyan = cv2.imread("goku_saiyan.png", -1)
peluca_payaso = cv2.imread("peluca_payaso.png", -1)
nariz_payaso = cv2.imread("nariz_payaso.png", -1)

def align_image_to_face(peluca, point1, point2, point3, face_point1, face_point2, face_point3):
    # Definir tres puntos de referencia en la imagen de la peluca
    peluca_points = np.float32([point1, point2, point3])
    face_points = np.float32([face_point1, face_point2, face_point3])

    # Calcular la transformación afín
    transformation_matrix = cv2.getAffineTransform(peluca_points, face_points)

    # Aplicar la transformación afín al tamaño del `frame` para cubrir toda la cara
    aligned_peluca = cv2.warpAffine(peluca, transformation_matrix, (frame.shape[1], frame.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

    return aligned_peluca

def is_mouth_open(landmarks, threshold=10):
    # Calcular la distancia entre el labio superior (punto 62) y el labio inferior (punto 66)
    top_lip = landmarks.part(62).y
    bottom_lip = landmarks.part(66).y
    mouth_open_distance = abs(bottom_lip - top_lip)
    
    # Verificar si la distancia supera el umbral para determinar si la boca está abierta
    return mouth_open_distance > threshold

filter = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detectar caras en el marco
    faces = detector(frame)

    for face in faces:
        # Obtener landmarks faciales
        landmarks = predictor(frame, face)

        # Obtener tres puntos de referencia en la cara
        forehead_left = (landmarks.part(0).x, landmarks.part(0).y)    # Esquina izquierda de la frente
        forehead_right = (landmarks.part(16).x, landmarks.part(16).y) # Esquina derecha de la frente
        chin = (landmarks.part(8).x, landmarks.part(8).y)             # Punto de la barbilla

        # Definir tres puntos en la peluca para alinear
        peluca_left = (150, 300)                    # Punto en el borde izquierdo de la peluca
        peluca_right = (350, 300)                   # Punto en el borde derecho de la peluca
        peluca_chin = (250, 440)                    # Punto en la parte inferior de la peluca (centro)

        if filter == 0:
            # Determinar si la boca está abierta
            if is_mouth_open(landmarks):
                peluca_actual = goku_saiyan  # Cambiar a la peluca de Goku Super Saiyan si la boca está abierta
            else:
                peluca_actual = goku_base  # Usar la peluca base si la boca está cerrada
            # Alinear la peluca con los puntos de la cara
            aligned_peluca = align_image_to_face(peluca_actual, peluca_left, peluca_right, peluca_chin, forehead_left, forehead_right, chin)

            # Componer la peluca en el marco principal
            alpha_peluca = aligned_peluca[:, :, 3] / 255.0
            for c in range(0, 3):
                frame[:, :, c] = (alpha_peluca * aligned_peluca[:, :, c] + (1 - alpha_peluca) * frame[:, :, c])

        elif filter == 1:
            # Obtener las coordenadas de los puntos de referencia
            point1 = (landmarks.part(36).x, landmarks.part(36).y)  # Esquina exterior del ojo izquierdo
            point2 = (landmarks.part(45).x, landmarks.part(45).y)  # Esquina exterior del ojo derecho
            point3 = (landmarks.part(57).x, landmarks.part(57).y)  # Centro del labio inferior

            # Calcular el centro y el radio para la región circular, con un margen extra
            center_x = int((point1[0] + point2[0] + point3[0]) / 3)
            center_y = int((point1[1] + point2[1] + point3[1]) / 3)
            radius = int(max(np.linalg.norm(np.array(point1) - np.array(point2)), 
                             np.linalg.norm(np.array(point1) - np.array(point3)),
                             np.linalg.norm(np.array(point2) - np.array(point3))) * 0.75)  # Aumentar tamaño

            # Crear una máscara circular y aplicar un desenfoque en los bordes
            mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
            cv2.circle(mask, (center_x, center_y), radius, 255, -1)
            mask = cv2.GaussianBlur(mask, (51, 51), 0)  # Aumentar el tamaño del kernel para mayor difuminado

            # Extraer la región circular de la cara
            x, y, w, h = center_x - radius, center_y - radius, radius * 2, radius * 2
            circular_region = cv2.bitwise_and(frame, frame, mask=mask)[y:y+h, x:x+w]

            # Aplicar el efecto espejo (voltear horizontalmente) y luego rotar 180 grados
            mirrored_region = cv2.flip(circular_region, 1)
            rotated_region = cv2.rotate(mirrored_region, cv2.ROTATE_180)

            # Aplicar la región rotada sobre el frame usando la máscara difuminada
            for c in range(3):  # Para cada canal de color
                frame[y:y+h, x:x+w, c] = (rotated_region[:, :, c] * (mask[y:y+h, x:x+w] / 255) +
                                          frame[y:y+h, x:x+w, c] * (1 - mask[y:y+h, x:x+w] / 255))
                
        elif filter == 2:
            # Obtener la coordenada del punto de la nariz
            nariz_x = landmarks.part(30).x
            nariz_y = landmarks.part(30).y

            # Aumentar el tamaño de la nariz de payaso
            ancho_nariz, alto_nariz = 100, 100  # Tamaño más grande para la nariz de payaso
            nariz_resized = cv2.resize(nariz_payaso, (ancho_nariz, alto_nariz))

            # Calcular la posición superior izquierda para centrar la nariz en el landmark
            x_offset = nariz_x - ancho_nariz // 2
            y_offset = nariz_y - alto_nariz // 2

            # Asegurarse de que la nariz encaje en el marco
            for c in range(3):  # Para cada canal de color
                # Aplicar transparencia usando el canal alfa
                frame[y_offset:y_offset+alto_nariz, x_offset:x_offset+ancho_nariz, c] = (
                    nariz_resized[:, :, c] * (nariz_resized[:, :, 3] / 255.0) +
                    frame[y_offset:y_offset+alto_nariz, x_offset:x_offset+ancho_nariz, c] * (1 - nariz_resized[:, :, 3] / 255.0)
                )
                
            # Alinear la peluca con los puntos de la cara
            aligned_peluca = align_image_to_face(peluca_payaso, peluca_left, peluca_right, peluca_chin, forehead_left, forehead_right, chin)

            # Componer la peluca en el marco principal
            alpha_peluca = aligned_peluca[:, :, 3] / 255.0
            for c in range(0, 3):
                frame[:, :, c] = (alpha_peluca * aligned_peluca[:, :, c] + (1 - alpha_peluca) * frame[:, :, c])

    # Mostrar el marco con el filtro aplicado
    cv2.imshow('Filtros', frame)

    key = cv2.waitKey(1)
    if key == 27:  # Presiona Esc para salir
        break

    elif key == ord('d'):  # Presiona "d" para alternar entre los filtros
        filter = (filter + 1) % 3  # Cambia entre 0 y 1

cap.release()
cv2.destroyAllWindows()
